In [ ]:
!apt-get update -y
!python3 -m pip install pandas
!python3 -m pip install plotly
!python3 -m pip install numpy==1.26.4
!python3 -m pip install matplotlib
!python3 -m pip install ipywidgets==8.1.2
!python3 -m pip install scipy
!python3 -m pip install umap umap-learn
!python3 -m pip install tqdm
!python3 -m pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np 
import os
import math
import gc
import sys
import warnings
from sklearn.exceptions import ConvergenceWarning
from sys import getsizeof
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from math import*
import plotly.express as px
from scipy.special import logsumexp
import ipywidgets as widgets
import threading
from plotly.colors import n_colors
from ipywidgets import interact,Layout,GridspecLayout, Button
from IPython.display import display, clear_output
from tqdm import tqdm
from scipy import sparse
from multiprocessing import Pool
from datetime import datetime
import concurrent.futures
from sklearn.random_projection import SparseRandomProjection
import plotly.offline as py
from matplotlib import pyplot as plt
import radviz_2d
py.init_notebook_mode(connected=True)
#import plotly.io as pio
%matplotlib inline

In [ ]:
#To check widgets are working
widgets.Button(
    description='Run',
    disabled=False,
    button_style='success',
)

In [ ]:
 def get_user_selection(a):
        with out:
            clear_output()
            start = datetime.now()
            select_country = dropbox1.value
            country_list= [str(x).split('.')[0] for x in dropbox1.value]
            country_name = [str(x).split('(')[0] for x in country_list]
            country= ','.join(sorted(country_name))
            print("country/countries selected: ",country)
            sample_size = dropbox3.value
            timestamp = dropbox4.value
            region = dropbox2.value
            lineage = dropbox6.value
            country_timestamp_region = [country,timestamp,region,lineage]
            c_t_r = ','.join(country_timestamp_region)
            country_len = len(select_country)
            sample_len = sample_size//country_len

            col_name =  ['Week','Host','Date','Country','Pangolin Clade','EPI_ID','x','y']
            timestamp_dict = {'oct-april_2023':'Last_6months','april2022-april2023':'Last_12months',
                              'april2021-april2024':'Last_24months','jan-jun_2020':'jan-jun_2020',
                              'jul-dec_2020':'jul-dec_2020', 'jan-dec_2020':'jan-dec_2020', 'jan-dec_2021':'jan-dec_2021','jan-jun_2021':'jan-jun_2021','jul-dec_2021':'jul-dec_2021',
                              'jan-jun_2022':'jan-jun_2022', 'jan-dec_2022':'jan-dec_2022', 'jul-dec_2022':'jul-dec_2022','jan-jun_2023':'jan_jun_2023','whole':'whole'}
            
            get_time_val = timestamp_dict[timestamp]
            print(get_time_val)
            def pick_timeline(t):
                if t == 'Last_6months':
                    idx=(meta[:,2]> '2022-10-17') & (meta[:,2]<='2023-04-17')
                    result= np.where(idx)[0]
                    return result
                   
                elif t == 'Last_12months':
                    idx=(meta[:,2]> '2022-04-17') & (meta[:,2]<='2023-04-17')
                    result=  np.where(idx)[0]
                    return result
                   
                elif t == 'Last_24months':
                    idx=(meta[:,2]> '2021-04-17') & (meta[:,2]<='2023-04-17')
                    result = np.where(idx)[0]
                    return result

                elif t == 'jan-jun_2020':
                    idx=(meta[:,2]> '2020-01-01') & (meta[:,2]<='2020-06-30')#2020-06-30
                    result= np.where(idx)[0]
                    return result

                elif t == 'jan-dec_2020':
                    idx=(meta[:,2]> '2020-01-01') & (meta[:,2]<='2020-12-31') #2021-06-30
                    result = np.where(idx)[0]
                    return result
                    
                elif t == 'jul-dec_2020':
                    print("hello")
                    idx=(meta[:,2]> '2020-07-01') & (meta[:,2]<='2020-12-31')
                    result=  np.where(idx)[0]
                    return result
                   
                elif t == 'jan-jun_2021':
                    idx=(meta[:,2]> '2021-01-01') & (meta[:,2]<='2021-06-30') #2021-06-30
                    result = np.where(idx)[0]
                    return result

                elif t == 'jan-dec_2021':
                    idx=(meta[:,2]> '2021-01-01') & (meta[:,2]<='2021-12-31') #2021-06-30
                    result = np.where(idx)[0]
                    return result
                  
                elif t == 'jul-dec_2021':
                    idx=(meta[:,2]> '2021-07-01') & (meta[:,2]<='2021-12-31')
                    result= np.where(idx)[0]
                    return result
            
                elif t == 'jan-jun_2022':
                    idx=(meta[:,2]> '2022-01-01') & (meta[:,2]<='2022-12-31') # 2022-06-30
                    result= np.where(idx)[0]
                    return result

                elif t == 'jul-dec_2022':
                    idx=(meta[:,2]> '2022-07-01') & (meta[:,2]<='2022-12-31')
                    result= np.where(idx)[0]
                    return result

                elif t == 'jan-dec_2022':
                    idx=(meta[:,2]> '2022-01-01') & (meta[:,2]<='2022-12-31') #2021-06-30
                    result = np.where(idx)[0]
                    return result

                elif t == 'jan-jun_2023':
                    idx=(meta[:,2]> '2023-01-01') & (meta[:,2]<='2023-06-30')
                    result= np.where(idx)[0]
                    return result
                    
                elif t == 'whole':
                    idx=(meta[:,2]>= '2020-01-01') & (meta[:,2]<='2022-06-30')
                    result= np.where(idx)[0]
                    return result
                   
                
            ######### reading the file/files, perform random sampling based on samplesize and putting them into dataframe######################3
            checkpoint =[]
            metadata = []
            for filename in tqdm(select_country,desc= "loading file"):
                new_list = []
                with open("meta_new/%s.tsv" % filename.split("(")[0]) as fin:
                    for line in fin:
                        line = line.rstrip().split("\t")
                        new_list.append(line)
                meta= np.array(new_list)
                #print(meta.shape)
                idx_meta = pick_timeline(get_time_val)
                if idx_meta.size == 0:
                    print("\033[1;31m No datapoints found for specified criteria  \n")
                    return
                

                index_meta = list(idx_meta)
                #print(len(index_meta))
                np.random.seed(42)
                sample_index_meta = np.random.choice(index_meta,min(len(index_meta),sample_len),replace =False)
                #print(sample_index_meta)
                with open("data_npz_new/%s.npz" % filename.split("(")[0], 'rb') as f:
                    data_matrix = sparse.load_npz(f)                    
                    #np.random.seed(42)
                    m = data_matrix[sample_index_meta]
                    X_m = m.toarray()
                    z_m = meta[sample_index_meta, :]
                    checkpoint.append(X_m)
                    metadata.append(z_m)
            X =np.concatenate(checkpoint)
            z = np.concatenate(metadata)
            #print(z)
            
            
           
            if dropbox6.value == 'All':
                
                X =np.concatenate(checkpoint)
                z = np.concatenate(metadata)

            elif dropbox6.value == 'Alpha':
                
                index_meta = np.where((z == 'B.1.1.7'))[0]
                #print(index_meta)
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis)
                
            elif dropbox6.value == 'Beta':
                
                index_meta = np.where((z == 'B.1.351'))[0]
                #print(index_meta)
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis)
            elif dropbox6.value == 'Gamma':
                
                index_meta = np.where((z == 'P.1'))[0]
                #print(index_meta)
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis)
            elif dropbox6.value == 'Delta':
                
                index_meta = np.where((z == 'B.1.617')|(z == 'B.1.617.1')|(z=='B.1.617.2')|(z=='B.1.617.3'))[0]
                #print(index_meta)
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis)
                #print(len(z))
            elif dropbox6.value == 'Omicron':
                
                #index_meta = np.where((z == 'BA.1')|(z== 'BA.1.1')|(z== 'BA.2')|(z== 'BA.3')|(z== 'BA.4')|(z== 'BA.5'))[0]
                index_meta = np.where((z== 'B.1.1.529')| (z == 'BA.1')|(z== 'BA.1.1')|(z== 'BA.2')|(z== 'BA.3')|(z== 'BA.4')|(z== 'BA.5'))[0] #In BOV Check once
                #print(index_meta)
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis)  
            elif dropbox6.value == 'VOC_All':
                
                index_meta = np.where((z == 'BA.1')|(z== 'B.1.1.7')|(z== 'BA.2')|(z== 'BA.3')|(z== 'BA.4')|(z== 'BA.5')|(z== 'B.1.617.2')|(z== 'P.1')|(z== 'B.1.351'))[0]
                #print(index_meta)
                #
                axis = 0
                z = np.take(z, index_meta, axis)
                X= np.take(X, index_meta,axis) 
            
                
                
            
            #print("Total memory used by loaded data: ",getsizeof(X))
            #################################################################################################

            ######## function to pick functional genomic region of interest#####################################
            def pick_region(v):
                if v == 'whole':
                    whole = X
                    return whole
                elif v == 'spike':
                    spike = np.array(X[:,55034:65385]).astype(np.int8)
                    return spike
                elif v == 'membrane':
                    membrane = np.array(X[:,68597:70341]).astype(np.int8)
                    return membrane
                elif v == 'envelope':
                    envelope = np.array(X[:,67849:68450]).astype(np.int8)
                    return envelope
                elif v == 'nucleocapsid':
                    nucleocapsid = np.array(X[:,73564:77025]).astype(np.int8)
                    return nucleocapsid
                return val
                
            
            val = pick_region(dropbox2.value)
            print("resulting vector after filtering",val.shape)

           
            print("Final dimension of feature vectors:", val.shape)
            if val.shape[0] == 0:
                print("\033[1;31m No datapoints found for specified criteria  \n")
                return
            #np.savetxt("foo.csv", val, fmt="%d", delimiter=",")
            
            flatten_arr = np.ravel(val)
            result = np.all(val==flatten_arr[0])
            if result :
                print("\033[1;31m There are no variation found in the specified selection, all datapoints are similar  \n")
                return
            '''def unique_rows(a):
                a = np.ascontiguousarray(a)
                unique_a = np.unique(a.view([('', a.dtype)]*a.shape[1]))
                return unique_a.view(a.dtype).reshape((unique_a.shape[0], a.shape[1]))'''
            '''def unique_rows(a):
                ind = np.lexsort(a.T)
                return a[np.concatenate(([True],np.any(a[ind[1:]]!= a[ind[:-1]],axis=1)))]
    

            
            uniq = unique_rows(val)
            print(len(uniq))'''
            
              
            #######################################################################################################
            
            ########### Perform chunking, normalization, dimensionality reduction on data########################
            RP_sparse= SparseRandomProjection(n_components=20,random_state=42)
            pca = PCA(n_components=2,random_state=42)
            def normalise(X_mod):
                row_sums=[]
                for i in range(0,X_mod.shape[0]):
                    rowsumval=np.sum(X_mod[i]**2)
                    row_sums.append(math.sqrt(rowsumval))

                rowsums=np.asarray(row_sums)
                np.seterr(divide='ignore', invalid='ignore')
                A_norm = np.divide(X_mod,rowsums[:,np.newaxis]).astype(np.float32) #row normalized input 
                A_norm[np.isnan(A_norm)] = 0
                return A_norm
                
            
            
            
            chunk_size = 2000
            total_size = val.shape[0]
            min_size = (min(total_size,sample_size))
            if min_size > 2000:
                if not min_size % 2000 == 0:
                    get_val = (min_size//chunk_size)+1
                else:
                    get_val = sample_size//chunk_size
                
            else:
                get_val = 1
            
            
            
            matrix=[]
            for i in tqdm(range(0, get_val),desc="Performing dimensionality reduction on feature vectors"):
                X_val = val[i*chunk_size : (i+1)*chunk_size]
                X_normalized = normalise(X_val)
                X_embed_partial= RP_sparse.fit_transform(X_normalized)
                del X_normalized
                gc.collect()
                matrix.append(X_embed_partial)
            X_embed = np.concatenate(matrix)


            #RP_sparse= SparseRandomProjection(n_components=15)
            #X_normalized = normalise(val)
            #X_embed = pca.fit_transform(X_normalized)
            #X_embed = RP_sparse.fit_transform(X_normalized)
            print("Dimension after dimensionality reduction on feature vectors: ", X_embed.shape)
 
            ##############################################################################################################################################################

            ######### Appropriate k val calculation if country is not present in dictionary#########################################################
           
            def k_val_max(val):
                no_of_clusters = [i for i in range(2,15)] 
                maxim=-1
                max_clust=-1
                scores=[]

                for n_cluster in tqdm(no_of_clusters,desc="Calculating appropriate k value"):
                    try:
                        with warnings.catch_warnings():
                            warnings.simplefilter("ignore", category=ConvergenceWarning)
                            cluster = KMeans(n_clusters = n_cluster)
                            cluster_labels = cluster.fit_predict(sample_kclust)
                            #print(cluster_labels)
                            silhouette_avg = silhouette_score(sample_kclust, cluster_labels)
                            scores.append(silhouette_avg)
                            if (silhouette_avg > maxim):
                                maxim=silhouette_avg
                                max_clust=n_cluster
                    except:
                        pass

                    
                return max_clust

        
                    

            sample_kclust = X_embed[np.random.choice(X_embed.shape[0],min(X_embed.shape[0],5000), replace=False), :]
            #print("dimension after random sampling for k value calculation: ",sample_kclust.shape)
            filename = "kval_lookup.txt"
            filesize = os.path.getsize(filename)

            if filesize == 0:
                max_clust = k_val_max(sample_kclust)
                country_timestamp_region.append(max_clust)
                with open(filename, 'w') as f:
                    f.write("\t".join(str(item) for item in country_timestamp_region))

            else:
                dictionary={}
                with open(filename) as f:
                    for line in f:
                        key, value = line.rstrip("\n").split("\t")
                        dictionary[key] = value

                    key = c_t_r
                    if key not in dictionary:
                        max_clust = k_val_max(sample_kclust)
                        if max_clust == -1:
                            max_clust
                        else:
                        
                            dictionary.update({key:max_clust})
                        #print("Dictionary: ",dictionary)
                        
                        with open(filename, 'w') as f:
                            for key, value in dictionary.items(): 
                                f.write('%s\t%s\n' % (key, value)) 

                    else:
                        print(" Optimal K value for selected criteria is: ",int(dictionary[key]))
                        max_clust = int(dictionary[key])
    

                   
            ####################################################################################################################
            ################# K-means clustering ##########################################################################
            if max_clust == -1:
                print("\033[1;31m  Selected samples had less variation in it, Clustering appointed all datapoints to similar cluster")
                return
                
            kmeans = KMeans(n_clusters = max_clust,random_state=42)
            kmeans.fit(X_embed)
            centroids  = kmeans.cluster_centers_
            labels = kmeans.labels_
            #print(labels)
            
            ##################################################################################################
            
            
            ######################################################################################################
            def ClusterIndicesNumpy(clustNum, labels_array): 
                #check = np.where(labels_array == clustNum)[0]
                #print("check_indices",len(check))
                return np.where(labels_array == clustNum)[0]
        
            
            def jaccard_similarity(argum):
                #print("HELLO")
                x = argum[0]
                y = argum[1]
                centroid_sum = argum[2]
                X_sum = argum[3]
                numerator = np.dot(x,y)
                #numerator = np.sum(np.minimum(x,y))
                denominator = (centroid_sum-numerator)+X_sum
                if denominator == 0:
                    return 1
                else:
                    return 1- (numerator/float(denominator))
                #return 1- (numerator/float(denominator))
            centroid_vector=[]
            
            centroid_sum=[]
            for i in tqdm(range(0, max_clust),desc="calculating centroid  vectors "):
                data = val[ClusterIndicesNumpy(i,kmeans.labels_)]
                centroid = data.mean(axis=0)
                #print("centroid of the following is : ",centroid)
                centroid_vector.append(centroid)
                centroid_sum.append(np.sum(centroid))
            #print(centroid_vector) 
            #np.savetxt("centroid.csv", centroid_vector, delimiter=",")

            def divide_chunks(l, n):
                for i in range(0, len(l), n):
                    yield l[i:i + n]
 
            max_threads = 6
            
            all_val_arr =[]
            jac_sim=[]
            #jac_sim_all=[]
            # Start processing the slides in parallel
            for j in range(0, len(X_embed)):
               
                X_sum= np.sum(val[j])
                
                for k in range(0, len(centroid_vector)):
                    all_val_arr.append((val[j],centroid_vector[k], centroid_sum[k],X_sum))
            #print(len(all_val_arr))
            
            with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
                for p in tqdm(executor.map(jaccard_similarity, all_val_arr),desc="Calculating weighted jaccard similarity"):
                    if p==None: continue
                    js = p
                    #print(js)
                    jac_sim.append(js)
                
                jac_sim_all = list(divide_chunks(jac_sim, len(centroid_vector)))
            jaccard_metric = jac_sim_all
            #print(jaccard_metric)
            jc_met = np.array(jaccard_metric)
            total_time_taken = datetime.now() - start
            print("Total time taken : ", total_time_taken)
            
            def plot(type_plot):
                if (type_plot == 'BOV_RadViz') | (type_plot == 'BOV_PCA'):
                    ptype = "Approach = BOVIZ"
                    

                    def print_plot(grpval):
                        pca = PCA(n_components=2)
                        components = pca.fit_transform(jaccard_metric)
                        #print(components)
                        map2 ={}
                        data = []

                        for i in range(0,len(z)):
                            map2[i] = z[i]
                        for k,v in enumerate(components):
                            meta = map2[k].tolist()
                            comp = v.tolist()
                            val = meta + comp
                            data.append(val)

                        df1 =pd.DataFrame(data,columns = col_name)
                        country_name_com = list(df1['Country'].unique())
                        plotname =  ",".join([country_name_com[i] for i in range(len(country_name_com))])+";"+ptype
                        #colors_data = pd.read_csv("Color_map.txt",sep='\t')
                        #group_color = colors_data.set_index('Clade')['RGB'].to_dict()
                        group_color = {'BA.4':'yellow','BA.2':'pink','BA.5':'violet','BA.3':'brown','B.1.351':'purple','P.1':'orange',
                                       'BA.1':'green','B.1.617.2':'red','B.1.1.7':'blue','India':'green','South Africa':'blue','Brazil':'orange',
                                      'United Kingdom': 'red'}
                        df1['counts'] = df1['Pangolin Clade'].map(df1['Pangolin Clade'].value_counts())
                        #print(df1['counts'])
                        df1= df1.sort_values(by=['counts'], ascending=False)
                        #print(df1)
                        df1["Week"] = df1["Week"].astype(float) #convert back to numeric
                        labels = {
                            str(i): f"PC {i+1} ({var:.1f}%)"
                            for i, var in enumerate(pca.explained_variance_ratio_ * 100)
                                }
                        #print(labels) 
                        #color_discrete_map=group_color
                        new = {'BA.4':'BA.4','BA.2':'BA.2(Omicron)','BA.5':'BA.5','BA.3':'BA.3','B.1.351':'B.1.351(Beta)','P.1':'P.1(Gamma)',
                                       'BA.1':'BA.1','B.1.617.2':'B.1.617.2(Delta)','B.1.1.7':'B.1.1.7(Alpha)'}
                        fig = px.scatter(df1, x='x', y='y',color=grpval,color_discrete_map=group_color,hover_name='EPI_ID', 
                                         hover_data= {'x' : False,'y' : False,'Week': True,'Host': True,'Date': True,'Country': True,'Pangolin Clade':True}
                                         ,labels=labels,width=1000, height=600)
                        #fig.for_each_trace(lambda t: t.update(name = new[t.name]))
                        fig.update_traces(marker=dict(size=10),
                                                  selector=dict(mode='markers'))
                        fig.update_layout(
                                    title=plotname, #plot title 
                                    paper_bgcolor='white',
                                    plot_bgcolor='white',
                                    showlegend = True,
                                    font=dict(
                                        family="Courier New, monospace",
                                        size=18,
                                        color="black"
                                    )
                                )
                        fig.update_xaxes(title_text=' ',
                                       showgrid=True,
                                       zeroline=True,
                                       showline=True,
                                       gridcolor='#bdbdbd',
                                       gridwidth=2,
                                       zerolinecolor='#969696',
                                       zerolinewidth=4,
                                       linecolor='#969696',
                                       linewidth=4)
                        fig.update_yaxes(title_text=' ',
                                       showgrid=True,
                                       zeroline=True,
                                       showline=True,
                                       gridcolor='#bdbdbd',
                                       gridwidth=2,
                                       zerolinecolor='#969696',
                                       zerolinewidth=4,
                                       linecolor='#969696',
                                       linewidth=4)

                        py.iplot(fig)
                        #py.plot(fig, filename='check.html')
                        
                    im = interact(print_plot,  grpval= ['Country','Pangolin Clade','Week'])
                    im.widget.children[0].description = 'Color code by : '
                    im.widget.children[0].style = {'description_width': 'initial'}
                    display(im)
                
                elif type_plot == 'radviz':
                    def print_plot_radviz(grpval):
                        ptype = "Approach = BOV RadViz"
                        plotname =  ",".join(str(x).split('(')[0] for x in country_list)+";"+ptype+";"+"k="+str(max_clust)

                        map_1 ={}
                        data = []
                        column_name =  ['Week','Host','Date','Country','Pangolin Clade','EPI_ID']
                        field = list(range(0,max_clust))
                        column_name.extend(field)
                        for i in range(0,len(z)):
                            map_1[i] = z[i]
                        for k,v in enumerate(jc_met):
                                meta = map_1[k].tolist()
                                comp = v.tolist()
                                val = meta + comp
                                data.append(val)

                        df1 =pd.DataFrame(data,columns = column_name)

                        df1['counts'] = df1['Pangolin Clade'].map(df1['Pangolin Clade'].value_counts())
                        df1= df1.sort_values(by=['counts'], ascending=False)
                        df1["Week"] = df1["Week"].astype(float) #convert back to numeric

                        y=df1[grpval]

                        X=df1.iloc[:,6:-1]
                        #print(X)
                        BPs=1000

                        radviz_2d.RadViz2D(y,X,BPs)
                    im = interact(print_plot_radviz,  grpval= ['Country','Pangolin Clade','Week'])
                    im.widget.children[0].description = 'Color code by : '
                    im.widget.children[0].style = {'description_width': 'initial'}
                    display(im)

            print("max_clust " ,max_clust)
            if max_clust > 15:
                if dropbox5.value == 'BOV_RadViz':
                    print("\033[1;32m The k value is more than specified range(k=10) for radviz, so we are running for PCA \n")
                    plot(dropbox5.value)
                elif dropbox5.value == 'BOV_PCA':
                    print("\033[1;32m we are running for PCA \n")
                    plot(dropbox5.value)
            
            elif max_clust <=15 and dropbox5.value == 'BOV_RadViz':
                print("\033[1;32m The selected choice will proceed for visualization with radviz \n")
                plot_name = 'radviz'
                plot(plot_name)
                
                
            elif max_clust <=15 and dropbox5.value == 'BOV_PCA':
                print("\033[1;32m The selected choice will proceed for visualization with PCA \n")
                plot(dropbox5.value)

In [ ]:
country_data = pd.read_csv("country_data_analysis.txt",sep='\t')
d = country_data.set_index('Country')['Samples'].to_dict()

'''def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("Country selected: %s" %change['new'])

dropbox1.observe(on_change)'''

out = widgets.Output()
dropbox1 = widgets.SelectMultiple(options=['{}({})'.format(k,v) for k,v in d.items()],description='Country:', rows=8, disabled=False)

#(Maximum datapoints allowed: 15000)
dropbox2 = widgets.Dropdown(
    options=['envelope','membrane','nucleocapsid','spike','whole'],
    value='whole',
    rows=5,
    description='Genomic region of interest:',
    style={'description_width': 'initial'},
    disabled=False
)

dropbox3 = widgets.BoundedIntText(
    value=10000,
    max=15000,
    step=500,
    description='Datapoints on canvas:',
    style={'description_width': 'initial'},
    disabled=False
)

dropbox4 = widgets.Dropdown(
    options=['oct-april_2023','april2022-april2023','april2021-april2024', 'jan-dec_2020','jan-jun_2020','jul-dec_2020','jan-dec_2021','jan-jun_2021','jul-dec_2021', 'jan-dec_2022', 'jan-jun_2022','jul-dec_2022','jan-jun_2023','whole'],
    value='oct-april_2023',
    rows=4,
    #layout={'width': 'max-content'},
    description='Timestamp:',
    style={'description_width': 'initial'},
    disabled=False
)

dropbox5 = widgets.Dropdown(
    options=['BOV_PCA','BOV_RadViz'],
    value='BOV_PCA',
    rows=3,
    #layout={'width': 'max-content'},
    description='Type of plot',
    style={'description_width': 'initial'},
    disabled=False
)

dropbox6 = widgets.Dropdown(
    options=['Alpha','Beta','Gamma','Delta','Omicron','VOC_All','All'],
    value= 'All',
    rows=5,
    description='Select Lineage:',
    style={'description_width': 'initial'
          },
    disabled=False
)


def dropdown_eventhandler(change):
    with out:
        clear_output()
        if change['type'] == 'change' and change['name'] == 'value':
            country_list= [str(x).split('.')[0] for x in change.new]
            country_name = [str(x).split('(')[0] for x in country_list]
            country= ','.join(sorted(country_name))
            print("\033[1;32m Country/Countries selected: ", country)


box_layout = widgets.Layout(display = 'flex',
                flex_flow='column',
                align_items='stretch',
                width='60%',
                height = '60%'
                 )
# Button to click
select_button = widgets.Button(
    description='Run',
    disabled=False,
    layout = box_layout,
    button_style='success',
)

dropbox1.observe(dropdown_eventhandler, names='value')

select_button.on_click(get_user_selection)

grid = GridspecLayout(5, 3, height='300px', align_items = 'baseline')
grid[0, 1] = dropbox6
grid[:, 0] = dropbox1
grid[1, 1] = dropbox4
grid[0, 2] = dropbox3
grid[4, 1] = select_button
grid[1,2] = dropbox5
grid[2,1] = dropbox2


'''ui = widgets.HBox([dropbox1,dropbox4,dropbox2])
ui_1 = widgets.HBox([dropbox3,dropbox5])
'''

#ui_run = widgets.HBox(children=[select_button],layout=box_layout)
display(grid,out)
dropbox1.observe(dropdown_eventhandler, names='value')
#display(ui,ui_1,ui_run,out)